In [1]:
%env CXX=/home/adr/micromamba/envs/dev/bin/g++

env: CXX=/home/adr/micromamba/envs/dev/bin/g++


In [2]:
!/home/adr/micromamba/envs/dev/bin/g++ --version

g++ (conda-forge gcc 12.3.0-0) 12.3.0
Copyright (C) 2022 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
import nutpie
import pathlib
import arviz
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import posteriordb
import json
import cmdstanpy
import time
import fastprogress
import pandas as pd
import xarray as xr
import os
import bridgestan

In [4]:
nutpie.__version__

'0.8.0'

In [5]:
bridgestan.__version__

'2.1.2'

In [6]:
cmdstanpy.__version__

'1.1.0'

In [7]:
# This is the version of the python wrapper
posteriordb.__version__

'0.2.0'

In [8]:
db_path = "../../posteriordb/posterior_database/"
db = posteriordb.PosteriorDatabase(db_path)

In [9]:
# The version of the posteriordb itself (one commit after 0.4)
!git -C "{db_path}" rev-parse HEAD

8bc65e69f7304d6f69023ba59ef9c5a04a1074fd


In [10]:
import logging
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

In [11]:
# Install cmdstan if necessary...
cmdstanpy.install_cmdstan()

Installing CmdStan version: 2.32.2
Install directory: /home/adr/.cmdstan
CmdStan version 2.32.2 already installed


True

In [12]:
def store_trace(storage, name, kind, tr):
    tr.to_netcdf(f"{storage}/{name}_{kind}.nc")
    ess = arviz.ess(tr)
    ess.to_netcdf(f"{storage}/{name}_{kind}_ess.nc")

In [13]:
def sample_nutpie(posterior, n_tunes, *, storage, n_chains=10, overwrite=True):
    with open(posterior.data.file_path()) as file:
        data = json.load(file)
    code = posterior.model.code("stan")

    start = time.time()
    compiled = nutpie.compile_stan_model(code=code).with_data(**posterior.data.values())
    end = time.time()
    time_compile = end - start

    for n_tune in n_tunes:
        start = time.time()
        tr = nutpie.sample(
            compiled,
            chains=n_chains,
            seed=1,
            progress_bar=False,
            draws=1000,
            tune=n_tune,
            cores=12,
            store_unconstrained=True,
            store_gradient=True,
        )
        end = time.time()
        tr.posterior.attrs["sampling_time"] = end - start
        tr.posterior.attrs["compile_time"] = time_compile
        store_trace(storage.format(n_tune=n_tune), posterior.name, "nutpie", tr)

def sample_stan(posterior, n_tunes, *, n_chains=10, storage, overwrite=True):
    start = time.time()
    cmd_model = cmdstanpy.CmdStanModel(posterior.model.name, posterior.model.code_file_path("stan"), compile=False)
    cmd_model.compile(force=True)
    end = time.time()
    time_compile = end - start
    with open(posterior.data.file_path()) as file:
        data = json.load(file)

    for n_tune in n_tunes:
        start = time.time()
        trace = cmd_model.sample(data, chains=n_chains, save_warmup=True,
                                 show_progress=False, seed=1, iter_warmup=n_tune, iter_sampling=1000)
        end = time.time()
        tr = arviz.from_cmdstanpy(trace, save_warmup=True)
        tr.posterior.attrs["sampling_time"] = end - start
        tr.posterior.attrs["compile_time"] = time_compile
        store_trace(storage.format(n_tune=n_tune), posterior.name, "stan", tr)

In [14]:
storage = "/mnt/harddrives/data_labs/covadapt-paper/2023-08-07-model-zoo_{n_tune}"

In [15]:
n_tunes = [300, 400, 600, 1000]

In [16]:
for n_tune in n_tunes:
    !mkdir -p {storage.format(n_tune=n_tune)}

In [17]:
names = db.posterior_names()

In [18]:
skip = [
    'ecdc0401-covid19imperial_v2',
    'ecdc0401-covid19imperial_v3',
    'ecdc0501-covid19imperial_v2',
    'ecdc0501-covid19imperial_v3',
    'election88-election88_full',
    'hmm_gaussian_simulated-hmm_gaussian',
    'iohmm_reg_simulated-iohmm_reg',
    'mnist-nn_rbm1bJ100',
    'mnist_100-nn_rbm1bJ10',
    'prideprejudice_chapter-ldaK5',
    'prideprejudice_paragraph-ldaK5',
    'prostate-logistic_regression_rhs',
]

In [ ]:
traces_nutpie = []
traces_stan = []

ess_nutpie = []
ess_stan = []

times_nutpie = []
times_stan = []

model_names = []
failed_models = []
errors = []

for i, name in enumerate(fastprogress.progress_bar(names)):
    if name in skip:
        continue

    try:
        sample_nutpie(db.posterior(name), n_tunes, storage=storage, overwrite=True)
    except (ValueError, RuntimeError, IndexError) as error:
        errors.append(("nutpie", error))
        print(name)
        print(errors[-1])
        failed_models.append(("nutpie", name))
    try:
        sample_stan(db.posterior(name), n_tunes, storage=storage, overwrite=True)
    except (ValueError, RuntimeError, IndexError) as error:
        errors.append(("stan", error))
        print(name)
        print(errors[-1])
        failed_models.append(("stan", name))

/home/adr/git/arviz/arviz/utils.py:185: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  numba_fn = numba.jit(**self.kwargs)(self.function)


hudson_lynx_hare-lotka_volterra
('stan', RuntimeError("Error during sampling:\nException: integrate_ode_rk45: initial state[1] is inf, but must be finite! (in '/home/adr/git/posteriordb/posterior_database/models/stan/lotka_volterra.stan', line 32, column 2 to line 35, column 42)\n\tException: integrate_ode_rk45: initial state[1] is inf, but must be finite! (in '/home/adr/git/posteriordb/posterior_database/models/stan/lotka_volterra.stan', line 32, column 2 to line 35, column 42)\n\tException: lognormal_lpdf: Location parameter[1] is -nan, but must be finite! (in '/home/adr/git/posteriordb/posterior_database/models/stan/lotka_volterra.stan', line 44, column 4 to column 47)\n\tException: lognormal_lpdf: Location parameter[1] is -nan, but must be finite! (in '/home/adr/git/posteriordb/posterior_database/models/stan/lotka_volterra.stan', line 44, column 4 to column 47)\n\tException: lognormal_lpdf: Location parameter[1] is -nan, but must be finite! (in '/home/adr/git/posteriordb/posterior_

arviz - WARNING - Array contains NaN-value.


sir-sir
('nutpie', ValueError("Sampling failed: Failed during evaluation: param_constrain() failed with exception: Exception: model_model_namespace::write_array: y[19][4] is -5.17146e-08, but must be greater than or equal to 0.000000 (in '/tmp/tmpp21cziwd/model.stan', line 52, column 2 to column 26)\n"))
